## what do
### create locations, input.mobility, and input.trace file
#### for each point:
##### 1. put x,y in locations
##### 2. put id (just number!!!) with time and location id in input.trace
#### create random 0-1-matrix (for now 50)

In [45]:
import sys
import pandas as pd
import numpy as np
import csv
import itertools  
from sklearn.preprocessing import normalize
np.set_printoptions(threshold=sys.maxsize)


In [61]:
df = pd.read_csv('trajectories.csv')
grouped = df.groupby('id', group_keys=False, sort=False).apply(lambda x: x.sort_values(by = 'timestamp', ascending=True))
df = grouped.reset_index(drop=True)
df
#df = df.groupby('id', sort=False).apply(lambda x: x.sort_values('timestamp')).reset_index(drop=True)
#df.to_csv('trajectories.csv', index=False)
#df[['x','y']].to_csv('locations', index=False, header=False)

,id,timestamp,x,y
0,100827,3.99955,-2.24235,-3.53080
1,100827,4.03163,-2.21459,-3.50224
2,100827,4.06292,-2.18689,-3.47377
3,100827,4.09167,-2.15925,-3.44542
4,100827,4.11919,-2.13169,-3.41719
...,...,...,...,...
59675,9954,11.99892,1.72422,4.47977
59676,9954,12.03413,1.73585,4.50835
59677,9954,12.06638,1.74748,4.53697
59678,9954,12.09895,1.75911,4.56564


In [ ]:
#NEW timestamps
time = list(range(1, max(df.groupby('id'))+1))
locs = []

In [3]:
time = [x+1 for x in df['timestamp'].sort_values(ascending=True).index]
time_trans = list(range(1, df.size+1))
time_dict = dict(zip(time, time_trans))

In [4]:
traces = []
current = ""
with open('trajectories.csv', "r") as traj:
    line_number = 0
    loc_number = 1
    for line in traj.readlines()[1:]:
        data = line.split(",")
        if data[0] != current:
            line_number += 1
            current = data[0]
        traces.append({
            "id": line_number,
            "timestamp": time_dict.get(loc_number),
            "location": loc_number,
        })
        loc_number += 1

In [5]:
users = range(traces[0]['id'], (traces[-1]['id'] + 1))
user_traces = [[x for x in traces if x['id'] == i] for i in users]
min_max_loc = [{'id': i, 'min_loc': min(user_traces[i-1], key=lambda x:x['location']).get('location'), 'max_loc': max(user_traces[i-1], key=lambda x:x['location']).get('location')} for i in users]

In [6]:
result = pd.DataFrame(traces, columns=['id', 'timestamp', 'location'])

In [10]:
max_time = max(traces, key=lambda x:x['timestamp']).get('timestamp')
max_user = max(traces, key=lambda x:x['id']).get('id')
max_loc = max(traces, key=lambda x:x['location']).get('location')
for i in range(1, max_user+1):
    timestamps_i = [x.get('timestamp') for x in user_traces[i-1]] #get timestamps from points of i
    locations_i = [x.get('location') for x in user_traces[i-1]] #...and the locations too
    timestamps_not_i = [*range(1, timestamps_i[0]), *range(timestamps_i[-1]+1,max_time+1)] #get timestamps where i hasn't started/already finished
    timestamps_otw_i = range(timestamps_i[0], timestamps_i[-1]+1) #get timestamps where i is already on the way
    df_standby_i = pd.DataFrame([{'id': i, 'timestamp': j, 'location': max_loc+1} for j in timestamps_not_i], columns=('id', 'timestamp', 'location'))
    df_otw_i = pd.DataFrame([[i, j, locations_i[timestamps_i.index(k)]] for j in timestamps_otw_i for k in timestamps_i if k < j and j < timestamps_i[timestamps_i.index(k)+1]], columns=('id', 'timestamp', 'location'))
    result.append(df_standby_i)
    result.append(df_otw_i)
result = result.groupby('id', group_keys=False, sort=False).apply(lambda x: x.sort_values(by = 'timestamp', ascending=True)).reset_index(drop=True)
result.to_csv('input.trace', index=False, header=False)

In [12]:
first_move = min([trace for user_trace in user_traces for trace in user_trace], key=lambda x:x['timestamp']).get('location')
last_move = max([trace for user_trace in user_traces for trace in user_trace], key=lambda x:x['timestamp']).get('location')
print(first_move)
print(last_move)
file = open('input.mobility',"w")
to_write = ''
for i in range(max_loc+1): #0 bis 59680, aber in echt 1 bis 59681
    if i != max_loc:
        line = np.zeros(max_loc+1) #array of 0 w length 59681
    else:
        line = np.ones(max_loc+1) #array of 1 w length 59681
        line[first_move-1] = 0 #except for the location, where user zu Zeitpunkt 1 schon im location ist.
    if i in [x['max_loc'] for x in min_max_loc] and i != last_move-1: # if that's the last location that a user visits and it's not the allerletzte move
        line[-1] = 1
    else: 
        line[i] = 1
        line[i+1] = 1
    line_to_write = np.array2string(line, separator=',')[1:-1] + '\n'
    print(line_to_write)
    to_write += line_to_write
file.write(to_write)
file.close()

13456
36016
1.,1.,0.,...,0.,0.,0.

0.,1.,1.,...,0.,0.,0.

0.,0.,1.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.


0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

...

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,...,0.,0.,0.

0.,0.,0.,..

#### create smaller version(s) of input data. . .

In [29]:
def get_sample(df, num):
    df_small = df.groupby('id', group_keys=False, sort=False).apply(lambda x: x.sort_values(by = 'timestamp', ascending=True).head(num)).head(500).reset_index(drop=True)
    df_small[['x','y']].to_csv('locations_'+str(num), index=False, header=False)
    with open("locations_"+str(num), "a") as file:
        file.write("11,11")
    file.close()
    users = pd.unique(df_small['id'])
    user_index = dict(zip(users, range(1, len(users)+1)))
    time = [x for x in df_small['timestamp'].sort_values(ascending=True)]
    time_trans = list(range(1, df_small.size+1))
    time_dict = dict(zip(time, time_trans))
    result = df_small[['id', 'timestamp']]
    result.replace({'id':user_index}, inplace=True)
    result.replace({'timestamp':time_dict}, inplace=True)
    result['location'] = np.resize(range(1, len(result)+1), len(result))
    result = result.astype('int64')
    return result
result_20 = get_sample(df, 5)
result_20['timestamp'].max()

/tmp/ipykernel_106020/2713170942.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['location'] = np.resize(range(1, len(result)+1), len(result))


100

In [39]:
def get_sample_new(df, user, loc):
    df_small = df.groupby('id', group_keys=False, sort=False).apply(lambda x: x.sort_values(by = 'timestamp', ascending=True).head(loc)).head(user*loc).reset_index(drop=True)
    df_small[['x','y']].to_csv('locations_'+str(user)+'_'+str(loc), index=False, header=False)
    users = pd.unique(df_small['id'])
    user_index = dict(zip(users, range(1, len(users)+1)))
    time = [x for x in df_small['timestamp']]
    time_trans = list(range(1,loc+1))*user
    print(time_trans)
    time_dict = dict(zip(time, time_trans))
    result = df_small[['id', 'timestamp']]
    result.replace({'id':user_index}, inplace=True)
    result.replace({'timestamp':time_dict}, inplace=True)
    result['location'] = np.resize(range(1, len(result)+1), len(result))
    result = result.astype('int64')
    result.to_csv('input.trace_'+str(user)+'_'+str(loc), index=False, header=False)
    return result
result = get_sample_new(df, 100, 5)
result

[1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 

/home/lia/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/tmp/ipykernel_106020/178693890.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['location'] = np.resize(range(1, len(result)+1), len(result))


,id,timestamp,location
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5
...,...,...,...
495,100,1,496
496,100,2,497
497,100,3,498
498,100,4,499


In [44]:
def get_mobility_new(df, user, loc):
    mobility = np.zeros((user*loc,user*loc))
    file = open('input.mobility_'+str(user)+'_'+str(loc),"w")
    to_write = ''
    for i in range(user*loc):
        if i+1 != user*loc:
            mobility[i][i+1] = 1
        line_to_write = ",".join(map(str,mobility[i]))+'\n'
        to_write += line_to_write
    file.write(to_write)
    file.close()
get_mobility_new(result, 100, 5)

In [30]:
def get_trace(df, location, num):
    max_time = max(df['timestamp'])
    print(max_time)
    max_user = max(df['id'])
    max_loc = max(df['location'])
    print(max_loc)
    for i in range(1, max_user+1):
        df_i = df[df['id'] == i]
        timestamps_i = df_i['timestamp'].to_list()
        timestamps_not_i = [*range(1, timestamps_i[0]), *range(timestamps_i[-1]+1,max_time+1)]
        timestamps_otw_i = range(timestamps_i[0], timestamps_i[-1]+1)                            
        df_standby_i = pd.DataFrame([[i, j, max_loc+1] for j in timestamps_not_i], columns=('id', 'timestamp', 'location'))
        df_otw_i = pd.DataFrame([[i, j, int(df_i[df_i['timestamp'] == k].location)] for j in timestamps_otw_i for k in timestamps_i if k < j and j < timestamps_i[timestamps_i.index(k)+1]], columns=('id', 'timestamp', 'location'))                                                         
        df = df.append(df_standby_i)
        df = df.append(df_otw_i)
    df = df.groupby('id', group_keys=False, sort=False).apply(lambda x: x.sort_values(by = 'timestamp', ascending=True)).reset_index(drop=True)
    df.to_csv('input.trace_'+str(num), index=False, header=False)
    return df
result_to_mob = get_trace(result_20, 'locations_5', 5)

100
100


In [31]:
def get_mobility(df, num):
    max_loc = df['location'].max() #max_loc schon +1
    first_move = int(df[(df['timestamp'] == 1) & (df['location'] != df['location'].max())].get('location'))
    last_move = int(df[(df['timestamp'] == max_loc-1) & (df['location'] != df['location'].max())].get('location'))
    id_max_loc = [{'id': i, 'max_loc': df[(df['location'] != max_loc) & (df['id'] == i)].iloc[-1].get('location')} for i in pd.unique(df['id'])]
    file = open('input.mobility_'+str(num),"w")
    to_write = ''
    for i in range(max_loc): #0 bis 59680, aber in echt 1 bis 59681
        if i != max_loc-1:
            line = np.zeros(max_loc) #array of 0 w length 59681
        else:
            line = np.ones(max_loc) #array of 1 w length 59681
            line[first_move-1] = 0 #except for the location, where user zu Zeitpunkt 1 schon im location ist.
        if i in [x['max_loc'] for x in id_max_loc] and i != last_move-1: # if that's the last location that a user visits and it's not the allerletzte move
            line[-1] = 1
        else: 
            line[i] = 1
            line[i+1] = 1
        line_to_write = ",".join(map(str,line))+'\n'
        to_write += line_to_write
    file.write(to_write)
    file.close()
    return to_write
result_to_mob.tail()
get_mobility(result_to_mob, 5)

'1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0